In [56]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from googlemaps import Client as GoogleMaps

MAX_AGE = 150
API_KEY = 'insert api key' #replace apyi key with a valid key
filename = "dataset_test.xlsx"

pd.options.display.float_format = '{:,.3f}'.format

In [57]:
xls = pd.ExcelFile(filename)

In [58]:
data = pd.read_excel(xls, 'data')

In [59]:
prices = pd.read_excel(xls, 'prices')

In [60]:
data.dtypes

Location                 object
First Name               object
Last Name                object
Birth Date       datetime64[ns]
Discount                float64
Entrance Time    datetime64[ns]
Exit Time        datetime64[ns]
dtype: object

In [61]:
# get age (full years) for each person
data['age'] = (data['Entrance Time'] - data['Birth Date']).astype('<m8[Y]')
# data.head()

In [62]:
# get number of hours spent in the park based on number of hours and minutes
data['hours'] = data['Exit Time'].dt.hour - data['Entrance Time'].dt.hour + (data['Exit Time'].dt.minute - data['Entrance Time'].dt.minute) / 60
data.head()

,Location,First Name,Last Name,Birth Date,Discount,Entrance Time,Exit Time,age,hours
0,Felix Water Park,Madalina,Derdena,1977-09-20,10.000,2019-07-27 13:29:18.258,2019-07-27 18:59:18.258,41.000,5.500
1,Therme Bucuresti,Stefania,Spataru,1983-09-03,nan,2019-06-08 14:45:12.223,2019-06-08 20:32:12.223,35.000,5.783
2,Therme Bucuresti,Petrut,Iliescu,2008-03-01,10.000,2019-08-03 10:50:51.364,2019-08-03 15:05:51.364,11.000,4.250
3,Aquapark Nymphaea,Mihaela,Derdena,1998-01-20,10.000,2019-08-18 08:04:15.376,2019-08-18 09:21:15.376,21.000,1.283
4,Aqua Magic Mamaia,Cristina,Cojocariu,1982-03-03,nan,2019-08-25 15:43:18.654,2019-08-25 17:15:18.654,37.000,1.533


In [63]:
# alternative method to compute hours, using timestamp
# data['Entrance Time Timestamp'] = data['Entrance Time'].apply(lambda x: datetime.timestamp(x))
# data['Exit Time Timestamp'] = data['Exit Time'].apply(lambda x: datetime.timestamp(x))
# data['Hours with Timestamp'] = (data['Exit Time Timestamp'] - data['Entrance Time Timestamp'])/60/60
# data.head()

In [64]:
# alternative method for retrieving information on prices for persons in data df (however it uses hardcoded figures)
# prices['Age Bucket'].unique()
# data['Age Bucket'] = data.age.map(lambda x: '0-3' if x <= 3 else ('4-13' if x <= 13 else ('14-18' if x <= 18 else ('19-50' if x <= 50 else '51+'))))
# merged = pd.merge(data, prices, on = ['Location', 'Age Bucket'], how = 'left')

In [65]:
# get min and max limits for each age bucket; consider max_age as limit for last age range
age_bucket_range = prices["Age Bucket"].str.split('[^0-9]', n = 1, expand = True) 
age_bucket_range[1] = age_bucket_range[1].replace('', MAX_AGE)

prices["Lower End Age Bucket"] = age_bucket_range[0].astype("int")
prices["Upper End Age Bucket"] = age_bucket_range[1].astype("int")
prices.head()

,Location,Age Bucket,Price,Address,Lower End Age Bucket,Upper End Age Bucket
0,Aquapark Nymphaea,0-3,0,"Aleea Ștrandului 13 B, Oradea 410051",0,3
1,Aquapark Nymphaea,4-13,5,"Aleea Ștrandului 13 B, Oradea 410051",4,13
2,Aquapark Nymphaea,14-18,8,"Aleea Ștrandului 13 B, Oradea 410051",14,18
3,Aquapark Nymphaea,19-50,10,"Aleea Ștrandului 13 B, Oradea 410051",19,50
4,Aquapark Nymphaea,51+,8,"Aleea Ștrandului 13 B, Oradea 410051",51,150


In [66]:
#retrieve information on age bucket and prices from prices df for persons in data df
merged = pd.merge(data, prices, on = ['Location'], how = 'left')
merged = merged[(merged['age'] >= merged['Lower End Age Bucket']) & (merged['age'] <= merged['Upper End Age Bucket'])]

In [67]:
#compute total fee to be paid by each person based on number of hours, price corresponding to age bucket and discount
merged['Discount'].fillna(0, inplace = True)
merged['Total fee'] = (merged['hours'] * merged['Price'] * (1 - merged['Discount'] / 100)).round(3)
merged.head()

,Location,First Name,Last Name,Birth Date,Discount,Entrance Time,Exit Time,age,hours,Age Bucket,Price,Address,Lower End Age Bucket,Upper End Age Bucket,Total fee
3,Felix Water Park,Madalina,Derdena,1977-09-20,10.000,2019-07-27 13:29:18.258,2019-07-27 18:59:18.258,41.000,5.500,19-50,10,"Strada Primăverii, Băile Felix 417500",19,50,49.500
8,Therme Bucuresti,Stefania,Spataru,1983-09-03,0.000,2019-06-08 14:45:12.223,2019-06-08 20:32:12.223,35.000,5.783,19-50,16,"Calea Bucureşti 1K, 077015",19,50,92.533
11,Therme Bucuresti,Petrut,Iliescu,2008-03-01,10.000,2019-08-03 10:50:51.364,2019-08-03 15:05:51.364,11.000,4.250,4-13,8,"Calea Bucureşti 1K, 077015",4,13,30.600
18,Aquapark Nymphaea,Mihaela,Derdena,1998-01-20,10.000,2019-08-18 08:04:15.376,2019-08-18 09:21:15.376,21.000,1.283,19-50,10,"Aleea Ștrandului 13 B, Oradea 410051",19,50,11.550
23,Aqua Magic Mamaia,Cristina,Cojocariu,1982-03-03,0.000,2019-08-25 15:43:18.654,2019-08-25 17:15:18.654,37.000,1.533,19-50,15,"DC86, Bulevardul Mamaia Nr. 334, Constanța 900001",19,50,23.000


In [68]:
#create column with full name and define export df
merged['Full Name'] = merged['First Name'] + ' ' + merged['Last Name']
export_df = merged[['Location', 'First Name', 'Last Name', 'Birth Date', 'Discount', 'Entrance Time', 'Exit Time', 'Total fee']]

In [69]:
#export_df.to_excel("full_dataset.xls", float_format = "%.3f", index = False) #formatting in Excel file not working as expected
writer = pd.ExcelWriter('full_dataset.xlsx', engine = 'xlsxwriter')
export_df.to_excel(writer, index = False, sheet_name = 'full_data')
workbook = writer.book
worksheet = writer.sheets['full_data']
format_fees = workbook.add_format({'num_format': '0.000'})
worksheet.set_column('H:H', None, format_fees)
writer.save()

In [70]:
#create columns with month and day of each entrance
merged['Month'] = merged['Entrance Time'].dt.month
merged['Day'] = merged['Entrance Time'].dt.day
merged['Month and Day'] = merged['Month'].astype('str') + '/' + merged['Day'].astype('str')
merged.head()

,Location,First Name,Last Name,Birth Date,Discount,Entrance Time,Exit Time,age,hours,Age Bucket,Price,Address,Lower End Age Bucket,Upper End Age Bucket,Total fee,Full Name,Month,Day,Month and Day
3,Felix Water Park,Madalina,Derdena,1977-09-20,10.000,2019-07-27 13:29:18.258,2019-07-27 18:59:18.258,41.000,5.500,19-50,10,"Strada Primăverii, Băile Felix 417500",19,50,49.500,Madalina Derdena,7,27,7/27
8,Therme Bucuresti,Stefania,Spataru,1983-09-03,0.000,2019-06-08 14:45:12.223,2019-06-08 20:32:12.223,35.000,5.783,19-50,16,"Calea Bucureşti 1K, 077015",19,50,92.533,Stefania Spataru,6,8,6/8
11,Therme Bucuresti,Petrut,Iliescu,2008-03-01,10.000,2019-08-03 10:50:51.364,2019-08-03 15:05:51.364,11.000,4.250,4-13,8,"Calea Bucureşti 1K, 077015",4,13,30.600,Petrut Iliescu,8,3,8/3
18,Aquapark Nymphaea,Mihaela,Derdena,1998-01-20,10.000,2019-08-18 08:04:15.376,2019-08-18 09:21:15.376,21.000,1.283,19-50,10,"Aleea Ștrandului 13 B, Oradea 410051",19,50,11.550,Mihaela Derdena,8,18,8/18
23,Aqua Magic Mamaia,Cristina,Cojocariu,1982-03-03,0.000,2019-08-25 15:43:18.654,2019-08-25 17:15:18.654,37.000,1.533,19-50,15,"DC86, Bulevardul Mamaia Nr. 334, Constanța 900001",19,50,23.000,Cristina Cojocariu,8,25,8/25


In [71]:
#Pivot table with total sales by location and month
pivot_sales_by_month = merged.pivot_table(index = 'Location', columns = 'Month', values = 'Total fee', aggfunc = np.sum)
pivot_sales_by_month.head()

Month,6,7,8
Location,,,
Aqua Magic Mamaia,"50,627.422","54,840.403","53,849.005"
Aquapark Nymphaea,"34,090.307","36,960.091","36,096.942"
Divertiland Water Park,"32,989.935","34,802.273","35,628.255"
Felix Water Park,"37,938.116","36,721.974","37,596.596"
Therme Bucuresti,"56,421.522","60,401.872","61,587.246"


In [72]:
#Pivot table with summarized info for locations, split on age bucket: number of visitors, average and total fee paid, average and total fees
pivot_sales_by_age = merged.pivot_table(index = ['Location', 'Age Bucket'], values = ['Full Name', 'hours', 'Total fee'], aggfunc = {'Full Name': len, 'hours': [np.mean, np.sum], 'Total fee': [np.mean, np.sum]})
pivot_sales_by_age

Full Name Total fee             hours  \
                                        len      mean         sum  mean   
Location               Age Bucket                                         
Aqua Magic Mamaia      14-18            532    31.727  16,878.545 3.265   
                       19-50           2677    47.213 126,389.075 3.232   
                       4-13             841    19.083  16,049.210 3.263   
Aquapark Nymphaea      14-18            550    25.341  13,937.654 3.267   
                       19-50           2542    31.786  80,799.146 3.270   
                       4-13             777    15.972  12,410.540 3.274   
Divertiland Water Park 14-18            551    16.160   8,904.099 3.326   
                       19-50           2576    31.941  82,281.104 3.281   
                       4-13             784    15.606  12,235.260 3.207   
Felix Water Park       14-18            558    21.966  12,257.077 3.217   
                       19-50           2741    31.820  87,219.235 3.265   
                       4-13             791    16.157  12,780.374 3.311   
Therme Bucuresti       14-18            571    38.185  21,803.600 3.272   
                       19-50           2656    50.817 134,969.236 3.259   
                       4-13             853    25.367  21,637.804 3.253   

                                             
                                        sum  
Location               Age Bucket            
Aqua Magic Mamaia      14-18      1,737.017  
                       19-50      8,652.867  
                       4-13       2,744.283  
Aquapark Nymphaea      14-18      1,796.633  
                       19-50      8,313.417  
                       4-13       2,544.150  
Divertiland Water Park 14-18      1,832.417  
                       19-50      8,452.983  
                       4-13       2,514.467  
Felix Water Park       14-18      1,794.833  
                       19-50      8,949.933  
                       4-13       2,618.717  
Therme Bucuresti       14-18      1,868.267  
                       19-50      8,654.583  
                       4-13       2,774.900

In [73]:
writer = pd.ExcelWriter('pivot_tables.xlsx', engine = 'xlsxwriter')
pivot_sales_by_month.to_excel(writer, sheet_name='Sales_by_month')
pivot_sales_by_age.to_excel(writer, sheet_name='Sales_by_age_summ')
writer.save()   


In [74]:
#dataframe split by month and day with counts
dfg1 = merged.groupby(["Month", "Day", "Month and Day"]).count().reset_index()
dfg1.head()

,Month,Day,Month and Day,Location,First Name,Last Name,Birth Date,Discount,Entrance Time,Exit Time,age,hours,Age Bucket,Price,Address,Lower End Age Bucket,Upper End Age Bucket,Total fee,Full Name
0,6,1,6/1,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120
1,6,2,6/2,216,216,216,216,216,216,216,216,216,216,216,216,216,216,216,216
2,6,3,6/3,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201,201
3,6,4,6/4,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182,182
4,6,5,6/5,224,224,224,224,224,224,224,224,224,224,224,224,224,224,224,224


In [75]:
#Line graphic with total daily number of visitors
fig_line = px.line(dfg1, x = "Month and Day", y = "Price", title = "Total daily number of visitors", labels={'Price':'Number of visitors'})
fig_line.show()

In [76]:
#Pie graphic with split of revenues on locations
fig_pie = px.pie(merged, values='Total fee', names='Location', title = 'Split of revenues on locations')
fig_pie.update_traces(textposition='inside', textinfo='percent+label')
fig_pie.show()

In [77]:
#Bar graphic with total sales on locations split by months
dfg_bar = merged.groupby(["Location", "Month"]).sum().reset_index()
dfg_bar["Month"] = dfg_bar["Month"].astype("str")
fig_bar = px.bar(dfg_bar, x = "Location", y = "Total fee", hover_data = ["Total fee"], color = "Month", title = "Total sales on locations")
fig_bar.show()

In [78]:
# retrieve geolocation of parks
parks = pd.read_excel(xls, 'parks')
gmaps = GoogleMaps(API_KEY)
parks["Latitude"] = ""
parks["Longitude"] = ""
for x in range(len(parks)):
    geocode_result = gmaps.geocode(parks['Address'][x])
    parks['Latitude'][x] = geocode_result[0]['geometry']['location'] ['lat']
    parks['Longitude'][x] = geocode_result[0]['geometry']['location']['lng']
#parks.to_excel("geolocation.xlsx")
parks.head()

,Location,Address,Latitude,Longitude
0,Aquapark Nymphaea,"Aleea Ștrandului 13 B, Oradea 410051",47.056,21.942
1,Aquapark Nymphaea,"Aleea Ștrandului 13 B, Oradea 410051",47.056,21.942
2,Aquapark Nymphaea,"Aleea Ștrandului 13 B, Oradea 410051",47.056,21.942
3,Aquapark Nymphaea,"Aleea Ștrandului 13 B, Oradea 410051",47.056,21.942
4,Aquapark Nymphaea,"Aleea Ștrandului 13 B, Oradea 410051",47.056,21.942
